**Day 10: Optimal Portfolio Theory (Markowitz) in Quantitative Finance**

### Concept: Optimal Portfolio Theory (Markowitz)
Optimal Portfolio Theory, developed by Harry Markowitz in 1952, is a foundational framework in quantitative finance for constructing portfolios that optimize the trade-off between expected return and risk. Also known as **Modern Portfolio Theory (MPT)**, it introduces the concept of diversification to reduce risk and uses the **mean-variance framework** to select portfolios that either maximize return for a given level of risk or minimize risk for a given level of return. The theory underpins portfolio management and asset allocation strategies.

### Technical Explanation
Markowitz’s framework assumes investors are rational and risk-averse, preferring higher returns and lower risk. The key metrics are:
- **Expected Return**: The weighted average of individual asset returns.
- **Risk**: Measured as the standard deviation (volatility) of portfolio returns, capturing uncertainty.
- **Diversification**: Combining assets with low or negative correlations reduces portfolio risk without necessarily sacrificing return.

For a portfolio of $ n $ assets with weights $ w = [w_1, \ldots, w_n] $, expected returns $ \mu = [\mu_1, \ldots, \mu_n] $, and covariance matrix $ \Sigma $, the portfolio’s expected return and variance are:

$$
\mu_p = w^T \mu = \sum_{i=1}^n w_i \mu_i
$$

$$
\sigma_p^2 = w^T \Sigma w = \sum_{i=1}^n \sum_{j=1}^n w_i w_j \sigma_{ij}
$$

Where $ \sigma_{ij} $ is the covariance between assets $ i $ and $ j $, and $ \sigma_{ii} = \sigma_i^2 $ is the variance of asset $ i $.

The **efficient frontier** is the set of portfolios that offer the highest expected return for a given level of risk (or lowest risk for a given return). These portfolios are found by solving an optimization problem:

1. **Minimize Risk for a Target Return**:
   $$
   \min_w w^T \Sigma w
   $$
   Subject to:
   $$
   w^T \mu = \mu_{\text{target}}, \quad \sum_{i=1}^n w_i = 1, \quad w_i \geq 0 \text{ (optional, no short-selling)}
   $$

2. **Maximize Return for a Target Risk**:
   $$
   \max_w w^T \mu
   $$
   Subject to:
   $$
   \sqrt{w^T \Sigma w} \leq \sigma_{\text{target}}, \quad \sum_{i=1}^n w_i = 1, \quad w_i \geq 0
   $$

Alternatively, investors can maximize a **utility function** incorporating a risk-aversion parameter $ \lambda $:

$$
\max_w w^T \mu - \frac{\lambda}{2} w^T \Sigma w
$$
Subject to: $ \sum_{i=1}^n w_i = 1 $, and optionally $ w_i \geq 0 $.

The solution to these problems yields the **optimal portfolio weights**. The efficient frontier is plotted as a curve in the return-risk ($ \mu_p, \sigma_p $) plane, with portfolios above the frontier unattainable and those below suboptimal.

**Key Insights**:
- **Diversification**: The covariance matrix $ \Sigma $ captures how asset returns move together. Negative or low correlations reduce $ \sigma_p $.
- **Risk-free asset**: Including a risk-free asset (e.g., Treasury bills) leads to the **Capital Market Line (CML)**, where the optimal portfolio combines the risk-free asset and the **tangency portfolio** (highest Sharpe ratio: $ (\mu_p - r_f) / \sigma_p $).
- **Limitations**: MPT assumes normal returns, constant expected returns and covariances, and ignores transaction costs or taxes. Real-world extensions address these, e.g., Black-Litterman model or robust optimization.

### Example
Let’s construct an optimal portfolio for three assets using Markowitz’s mean-variance optimization. Parameters:
- Assets: Stock A, Stock B, Bond C
- Expected annual returns: $ \mu = [10\%, 15\%, 5\%] $
- Annual volatilities: $ \sigma = [20\%, 30\%, 10\%] $
- Correlation matrix:
  $$
  \rho = \begin{bmatrix}
  1 & 0.5 & 0.2 \\
  0.5 & 1 & 0.3 \\
  0.2 & 0.3 & 1
  \end{bmatrix}
  $$
- Portfolio value: $1,000,000
- Constraint: No short-selling ($ w_i \geq 0 $), weights sum to 1
- Goal: Minimize risk for a target return of 9% annually

**Step-by-Step**:
1. Construct the covariance matrix from volatilities and correlations.
2. Solve the quadratic optimization problem to find weights.
3. Calculate the portfolio’s expected return and risk.
4. Allocate the $1,000,000 across assets.

**Covariance Matrix**:
$$
\sigma_{ij} = \rho_{ij} \sigma_i \sigma_j
$$
$$
\Sigma = \begin{bmatrix}
0.20^2 & 0.5 \cdot 0.20 \cdot 0.30 & 0.2 \cdot 0.20 \cdot 0.10 \\
0.5 \cdot 0.20 \cdot 0.30 & 0.30^2 & 0.3 \cdot 0.30 \cdot 0.10 \\
0.2 \cdot 0.20 \cdot 0.10 & 0.3 \cdot 0.30 \cdot 0.10 & 0.10^2
\end{bmatrix}
= \begin{bmatrix}
0.04 & 0.03 & 0.004 \\
0.03 & 0.09 & 0.009 \\
0.004 & 0.009 & 0.01
\end{bmatrix}
$$

**Optimization**:
$$
\min_w w^T \Sigma w
$$
Subject to:
$$
w^T \mu = 0.09, \quad \sum w_i = 1, \quad w_i \geq 0
$$

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [ ]:
# Parameters
mu = np.array([0.10, 0.15, 0.05]) # Expected returns
sigma = np.array([0.20, 0.30, 0.10]) # Volatilities
rho = np.array([[1, 0.5, 0.2], [0.5, 1, 0.3], [0.2, 0.3, 1]]) # Correlation matrix
target_returns  = np.linspace(0.05, 0.15, 100) # Range of targets
optimal_target = 0.11 # Target return
portfolio_value = 1_000_000

# Covariance Matrix
Sigma = np.outer(sigma, sigma) * rho

In [ ]:
# Optimization Problem (No visualization)
w = cp.Variable(3) # Portfolio Weights
objective = cp.Minimize(cp.quad_form(w, Sigma)) # Minimize variance
constraints = [
    w.T @ mu == optimal_target, # Target return 
    cp.sum(w) == 1,            # Weights add up to 1
    w >= 0                     # No short selling
]
problem = cp.Problem(objective, constraints)
problem.solve()

# Results
weights = w.value
portfolio_return = weights @ mu
portfolio_vol = np.sqrt(weights @ Sigma @ weights)
allocations = weights * portfolio_value

print(f"Optimal Weights: Stock A = {weights[0]:.2%}, Stock B = {weights[1]:.2%}, Stock C = {weights[2]:.2%}")
print(f"Portfolio Expected Return: {portfolio_return:.2%}")
print(f"Portfolio Volatility (Annual): {portfolio_vol:.2%}")
print(f"Portfolio Allocations: Stock A: ${allocations[0]:,.0f}, Stock B: ${allocations[1]:,.0f}, Bond C: ${allocations[2]:,.0f}")

In [ ]:
# Compute efficient frontier
volatilities = []
returns = []
optimal_weights = None
optimal_vol = None

for target in target_returns:
    w = cp.Variable(3)
    objective = cp.Minimize(cp.quad_form(w, Sigma))
    constraints = [
        w.T @ mu == target,
        cp.sum(w) == 1,
        w >= 0
    ]
    problem = cp.Problem(objective, constraints)
    try:
        problem.solve()
        if problem.status == cp.OPTIMAL:
            vol = np.sqrt(w.value @ Sigma @ w.value)
            volatilities.append(vol)
            returns.append(target)
            if abs(target - optimal_target) < 1e-2:  # Store optimal portfolio at 11%
                optimal_weights = w.value
                optimal_vol = vol
    except:
        pass  # Skip infeasible solutions

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(volatilities, returns, 'b-', label='Efficient Frontier')
plt.scatter(sigma, mu, c='red', marker='o', s=50, label='Individual Assets')
plt.scatter(optimal_vol, optimal_target, c='green', marker='*', s=50, 
           label=f'Optimal Portfolio (Return={optimal_target:.2%})')
plt.xlabel('Portfolio Volatility (Standard Deviation)')
plt.ylabel('Portfolio Expected Return')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True)
plt.show()

# Print optimal portfolio details
if optimal_weights is not None:
    print(f"Optimal Portfolio at {optimal_target:.2%} Return:")
    print(f"Weights: Stock A: {optimal_weights[0]:.2%}, Stock B: {optimal_weights[1]:.2%}, Bond C: {optimal_weights[2]:.2%}")
    print(f"Volatility: {optimal_vol:.2%}")